In [1]:
import numpy as np
target="/special/jbpark/TabS6LWhole/train_dataset/"

left_eye_right_top = np.load(target+"left_eye_right_top.npy")
left_eye_left_bottom = np.load(target+"left_eye_left_bottom.npy")
right_eye_right_top = np.load(target+"right_eye_right_top.npy")
right_eye_left_bottom = np.load(target+"right_eye_left_bottom.npy")


left_eye_right_top[:,1] = left_eye_right_top[:,1] - left_eye_left_bottom[:,1]
left_eye_right_top[:,0] = left_eye_left_bottom[:,0] - left_eye_right_top[:,0]

right_eye_right_top[:,1] = right_eye_right_top[:,1] - right_eye_left_bottom[:,1]
right_eye_right_top[:,0] = right_eye_left_bottom[:,0] - right_eye_right_top[:,0]

In [2]:
left_eye_right_top

array([[0.14787375, 0.08312498],
       [0.13619948, 0.07656247],
       [0.15176517, 0.08531243],
       ...,
       [0.14398227, 0.08093749],
       [0.15954793, 0.08968751],
       [0.14009085, 0.07874998]])

In [3]:
right_eye_right_top

array([[0.14787374, 0.08312506],
       [0.1400909 , 0.07875004],
       [0.14787374, 0.08312502],
       ...,
       [0.13619948, 0.07656249],
       [0.15954793, 0.08968752],
       [0.14009085, 0.07874998]])

In [5]:
import json
import glob
import os
import random
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import concatenate, ZeroPadding2D, Add, add, Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.initializers import Constant
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import multi_gpu_model
import gc
from pathlib import Path

dropout_rate=0
resolution = 64
channels = 1
target="/special/jbpark/TabS6LWhole/train_dataset/"
model_dir = "/special/jbpark/EvalModel"

def custom_loss(y_true, y_pred): 
    #euclidean loss
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1)) # /205.7
    #squared loss
    #return K.sum(K.square(y_pred - y_true), axis=-1) 
    
SConv1 = Conv2D(32, (3, 3), padding = 'same', activation = 'relu')
SConv2 = Conv2D(64, (3, 3), padding = 'same', activation = 'relu')
SConv3 = Conv2D(64, (3, 3), padding = 'same', activation = 'relu')

# Left Eye
input1 = Input(shape=(64, 64,channels), name='left_eye')
x = SConv1(input1)
x = MaxPooling2D(pool_size = (2, 2))(x)

x = SConv2(x)
x = MaxPooling2D(pool_size = (2, 2))(x)

x = SConv3(x)
left_eye = MaxPooling2D(pool_size = (2, 2))(x)
left_eye = Flatten()(left_eye)

# Right Eye
input2 = Input(shape=(64, 64,channels), name='right_eye')
x = SConv1(input2)
x = MaxPooling2D(pool_size = (2, 2))(x)

x = SConv2(x)
x = MaxPooling2D(pool_size = (2, 2), padding='same')(x)

x = SConv3(x)
right_eye = MaxPooling2D(pool_size = (2, 2), padding='same')(x)
right_eye = Flatten()(right_eye)

# Eyes
eyes = concatenate([left_eye, right_eye])
fc1 = Dense(64, activation='relu')(eyes)
fc2 = Dense(16, activation='relu')(fc1)
fc2 = Dropout(rate=dropout_rate)(fc2)

# Facepos
input4 = Input(shape=(1, 1, 2), name='facepos')
facepos = Flatten()(input4)

#Euler
input3 = Input(shape=(1, 1, 3), name='euler')
euler = Flatten()(input3)

# Eye size
input5 = Input(shape=(1, 1, 2), name='left_eye_right_top')
input6 = Input(shape=(1, 1, 2), name='right_eye_right_top')
left_eye_right_top = Flatten()(input5)
right_eye_right_top = Flatten()(input6)
eye_corner = concatenate([left_eye_right_top, right_eye_right_top])

head_pose = concatenate([euler, facepos, eye_corner])
fc_f1 = Dense(16, activation='relu')(head_pose)

# FC2, FC3
fc2 = concatenate([fc2, fc_f1])
fc2 = Dense(16, activation='relu')(fc2)
fc3 = Dense(2, activation='linear', name='pred')(fc2)

fc3 = add([fc3,facepos])
pred = fc3

model = Model(inputs=[input1, input2, input3, input4, input5, input6], outputs=[pred])

tf.distribute.MirroredStrategy()

model.compile(loss=custom_loss, optimizer=Adam(lr=1e-3))
model.summary()

gaze_point = np.load(target+"gaze_point.npy").astype(float)
left_eye = np.load(target+"left_eye.npy").reshape(-1,resolution,resolution,channels)
right_eye = np.load(target+"right_eye.npy").reshape(-1,resolution,resolution,channels)
euler = np.load(target+"euler.npy").reshape(-1,1,1,3)
facepos = np.load(target+"facepos.npy").reshape(-1,1,1,2)
left_eye_right_top = np.load(target+"left_eye_right_top.npy")
left_eye_left_bottom = np.load(target+"left_eye_left_bottom.npy")
right_eye_right_top = np.load(target+"right_eye_right_top.npy")
right_eye_left_bottom = np.load(target+"right_eye_left_bottom.npy")


left_eye_right_top[:,1] = left_eye_right_top[:,1] - left_eye_left_bottom[:,1]
left_eye_right_top[:,0] = left_eye_left_bottom[:,0] - left_eye_right_top[:,0]

right_eye_right_top[:,1] = right_eye_right_top[:,1] - right_eye_left_bottom[:,1]
right_eye_right_top[:,0] = right_eye_left_bottom[:,0] - right_eye_right_top[:,0]

left_eye_right_top = left_eye_right_top.reshape(-1,1,1,2)
right_eye_right_top = left_eye_left_bottom.reshape(-1,1,1,2)

epoch = 1000
Path(model_dir+'/checkpoint').mkdir(parents=True, exist_ok=True)
mc = ModelCheckpoint(model_dir+'/checkpoint/gazel_shared_ver.h5', monitor='val_loss', mode='min', save_best_only=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
#hist = model.fit([left_eye,right_eye, euler, facepos],gaze_point, validation_split=0.1,epochs=epoch, callbacks=[es, mc])
hist = model.fit([left_eye, right_eye, euler, facepos, left_eye_right_top, right_eye_right_top,],gaze_point, validation_split=0.1,epochs=epoch, callbacks=[es, mc])

Path(model_dir+'/Lmodels').mkdir(parents=True, exist_ok=True)
model.save(model_dir+'/Lmodels/gazel_shared_ver.h5')

K.clear_session()
%xdel -n gaze_point
%xdel -n left_eye
%xdel -n right_eye
%xdel -n euler
%xdel -n facepos
%xdel -n left_eye_right_top
%xdel -n left_eye_left_bottom
%xdel -n right_eye_right_top
%xdel -n right_eye_left_bottom

gc.collect()
%reset -f

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
left_eye (InputLayer)           [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
right_eye (InputLayer)          [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 32)   320         left_eye[0][0]                   
                                                                 right_eye[0][0]                  
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 32, 32, 32)   0           conv2d[0][0]                 

##### if you get error with toco_protos or toco run the python code with terminal or convert whole file to .py file then run

In [1]:
import numpy as np
import tensorflow as tf
from pathlib import Path
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model


def custom_loss(y_true, y_pred): 
    #euclidean loss
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1)) # /205.7
    #squared loss
    #return K.sum(K.square(y_pred - y_true), axis=-1) 

model_dir = "/special/jbpark/TabletModel/RGBmodels/"
keras_file = model_dir+'gaze_estimator.h5'
# Convert to TensorFlow Lite model.
model = load_model(keras_file,custom_objects={'custom_loss': custom_loss})

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TF Lite model.
with tf.io.gfile.GFile(model_dir+'tflite/gaze_estimator.tflite', 'wb') as f:
    f.write(tflite_model)

ConverterError: See console for info.
/bin/sh: 1: toco_from_protos: not found




In [3]:
import glob

path = glob.glob("/special/jbpark/TabS6Model/checkpoint/tflite/*")
path.sort()
path

['/special/jbpark/TabS6Model/checkpoint/tflite/Seunghoon_mse_sage.tflite',
 '/special/jbpark/TabS6Model/checkpoint/tflite/Seunghoon_mysage.tflite',
 '/special/jbpark/TabS6Model/checkpoint/tflite/Seunghoon_sage.tflite',
 '/special/jbpark/TabS6Model/checkpoint/tflite/customloss_sage.tflite',
 '/special/jbpark/TabS6Model/checkpoint/tflite/facepos.tflite',
 '/special/jbpark/TabS6Model/checkpoint/tflite/fc_layer_opt.tflite',
 '/special/jbpark/TabS6Model/checkpoint/tflite/grid_mysage.tflite',
 '/special/jbpark/TabS6Model/checkpoint/tflite/illum_facepos.tflite',
 '/special/jbpark/TabS6Model/checkpoint/tflite/illum_mysage.tflite',
 '/special/jbpark/TabS6Model/checkpoint/tflite/mysage.tflite',
 '/special/jbpark/TabS6Model/checkpoint/tflite/noeuler_illum_mysage.tflite',
 '/special/jbpark/TabS6Model/checkpoint/tflite/rgb_facepos.tflite',
 '/special/jbpark/TabS6Model/checkpoint/tflite/sage.tflite',
 '/special/jbpark/TabS6Model/checkpoint/tflite/seunghoon_sage.tflite',
 '/special/jbpark/TabS6Model/

In [9]:
import tensorflow as tf
print(path[7])
tflite = tf.lite.Interpreter(model_path=path[7])
tflite.get_input_details()

/special/jbpark/TabS6Model/checkpoint/tflite/illum_facepos.tflite


[{'name': 'left_eye',
  'index': 4,
  'shape': array([ 1, 64, 64,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)},
 {'name': 'euler',
  'index': 1,
  'shape': array([1, 1, 1, 3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)},
 {'name': 'facepos',
  'index': 3,
  'shape': array([1, 1, 1, 2], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)},
 {'name': 'face_grid',
  'index': 2,
  'shape': array([ 1, 25, 25,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)},
 {'name': 'right_eye',
  'index': 56,
  'shape': array([ 1, 64, 64,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)}]